In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

import multiprocess
# with multiprocess.Pool(processes=8) as pool:
#   for i in pool.imap_unordered(f, a):
#     print(i)

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('17')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [2]:
lines_temp, groups_temp = linesplit("""
####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##
""".strip())

In [3]:
lines, groups = linesplit("""
>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
""".strip())

In [4]:
lines, groups = linesplit(raw_input, verbose=True)

1 lines
1 groups
>>>
>>>><<>>>><<><<<>>><<<>><<>>><<>>>><<<>>><<<>><<<<>>>><<<<>>>><<<>>>><<<<><><>>><<<<>><>>>><<<><<<>>...


In [5]:
minos = []

for group in groups_temp:
  ys = len(group)
  xs = len(group[0])
  mino = []
  
  for i in range(len(group)):
    for j in range(len(group[0])):
      if group[i][j] == '#':
        mino.append(Point(j, ys - i - 1))
  
  minos.append(mino)

minos

[[[+0, +0], [+1, +0], [+2, +0], [+3, +0]],
 [[+1, +2], [+0, +1], [+1, +1], [+2, +1], [+1, +0]],
 [[+2, +2], [+2, +1], [+0, +0], [+1, +0], [+2, +0]],
 [[+0, +3], [+0, +2], [+0, +1], [+0, +0]],
 [[+0, +1], [+1, +1], [+0, +0], [+1, +0]]]

In [6]:
width = 7

In [7]:
jets = lines[0]

mino_index = 0
jet_index = 0

stored = set()

def moved(points, delta):
  return [i + delta for i in points]

for i in range(2022):
  a = list(minos[mino_index])
  mino_index = (mino_index + 1) % len(minos)
  
  base_x = 2
  if stored:
    base_y = max([i.y for i in stored]) + 4
  else:
    base_y = 3
  
  a = [i + Point(base_x, base_y) for i in a]
  
  while True:
    aa = moved(a, LEFT if jets[jet_index] == '<' else RIGHT)
    jet_index = (jet_index + 1) % len(jets)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      pass
    else:
      a = aa
      
      
    aa = moved(a, DOWN)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      stored |= set(a)
      
      break
    else:
      a = aa

max([i.y for i in stored]) + 1

3163

In [47]:
submit1(_)

Submitting 3068...
[<p>You don't seem to be solving the right level.  Did you already complete it? <a href="/2022/day/17">[Return to Day 17]</a></p>]


/Users/abra/projects/jupyter/adventofcode2022/adventlib/common.py:51: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 51 of the file /Users/abra/projects/jupyter/adventofcode2022/adventlib/common.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  print(BeautifulSoup(response.text).select('main>article>p'))


AssertionError: 

In [8]:
len(minos), len(jets)

(5, 10091)

In [21]:
jets = lines[0]

mino_index = 0
jet_index = 0

stored = set()

deltas = []

def moved(points, delta):
  return [i + delta for i in points]

memo = {}

for it in tqdm(range(2022)):
  a = list(minos[mino_index])
  mino_index = (mino_index + 1) % len(minos)
  
  base_x = 2
  if stored:
    base_y = max([i.y for i in stored]) + 4
  else:
    base_y = 3
    
    
  bottom = []
  for x in range(0, 7):
    for y in range(base_y - 12, base_y):
      p = Point(x, y)
      if p in stored:
        bottom.append(Point(x, base_y - y))
  
  state = (
    mino_index,
    jet_index,
    tuple(bottom)
  )
  
  # if stored:
  #   if (it - magic1) % magic2 == 0:
  #     print(it, max([i.y for i in stored]) + 1 + 0)
  
  if state in memo:
    print(memo[state])
    print((it, max([i.y for i in stored])))
    print(state)
    magic1 = memo[state][0]
    magic2 = it - memo[state][0]
    gain = max([i.y for i in stored]) - memo[state][1]
    break
  else:
    if stored:
      memo[state] = (it, max([i.y for i in stored]))
  
  a = [i + Point(base_x, base_y) for i in a]
  
  while True:
    aa = moved(a, LEFT if jets[jet_index] == '<' else RIGHT)
    jet_index = (jet_index + 1) % len(jets)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      pass
    else:
      a = aa
      
      
    aa = moved(a, DOWN)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      stored |= set(a)
      print(min([base_y - i.y for i in aa]))
      deltas.append(min([base_y - i.y for i in aa]))
      break
    else:
      a = aa

max([i.y for i in stored]) + 1, magic1, magic2, gain

  0%|          | 0/2022 [00:00<?, ?it/s]

4
2
2
3
12
6
3
2
3
3
4
2
3
23
21
4
2
2
11
9
6
3
2
3
11
6
3
3
1
9
8
3
2
12
5
4
2
3
3
5
4
2
3
1
3
4
2
2
17
3
4
3
2
6
4
5
2
2
18
7
4
2
3
6
4
4
2
5
3
3
8
5
2
1
9
4
3
4
3
6
4
2
2
1
9
4
2
2
20
5
4
2
5
2
8
4
2
2
3
3
4
2
5
1
7
4
2
3
21
8
6
3
3
8
3
4
2
2
1
13
4
2
2
16
9
4
3
4
2
3
4
2
3
1
9
4
3
2
1
3
4
2
3
3
7
4
2
3
15
3
8
5
2
7
5
4
2
2
12
5
4
3
2
6
4
4
2
3
1
9
8
3
7
5
3
4
2
2
1
3
4
2
2
1
3
12
2
12
10
8
4
3
2
10
5
4
2
2
7
3
4
2
2
7
3
4
2
2
12
14
4
2
2
17
3
4
2
5
3
3
8
3
3
5
3
6
2
6
2
3
4
2
3
1
9
8
3
2
3
13
4
2
5
1
8
8
5
2
7
5
6
3
2
1
9
4
3
3
3
5
4
2
2
3
7
4
2
5
1
7
6
3
2
7
5
4
2
2
7
5
4
3
4
2
8
4
2
3
3
10
4
2
2
7
3
4
2
2
3
3
4
2
2
3
3
4
3
4
3
3
4
2
2
13
5
4
3
3
3
8
4
2
2
7
3
4
2
2
3
16
10
3
2
1
9
4
7
2
3
11
4
2
3
3
3
10
7
2
9
6
6
3
2
7
5
4
2
5
1
7
4
2
5
1
9
4
2
3
4
4
4
2
3
3
3
4
3
2
3
5
4
8
2
3
3
4
2
2
18
5
4
2
3
1
9
4
3
3
5
3
4
2
2
19
5
4
2
3
13
3
8
3
3
3
7
6
3
2
7
5
4
3
2
1
3
4
3
3
3
3
4
3
4
13
5
4
3
2
6
5
4
2
3
7
5
4
2
2
3
3
4
2
5
3
3
4
8
2
4
3
4
3
4
3
7
4
2
3
1
9
10
7
2
3
12
4
2
3
1
9
4
3
4


(2922, 153, 1715, 2677)

In [22]:
max(deltas)

62

In [83]:
magic1 = 23
magic2 = 58 - 23

gain = 94 - 41

In [104]:
magic1 = 158
magic2 = 1873 - 158

gain = 2928 - 251

In [107]:
jets = lines[0]

mino_index = 0
jet_index = 0

stored = set()

def moved(points, delta):
  return [i + delta for i in points]

memo = {}

goal = 1000000000000
it = 0
cheats = (goal - magic1) // magic2
closest_goal = cheats * magic2 + magic1
gained_by_magic = 0

for i in tqdm(range(1000000)):
  if it >= goal:
    break
    
  if it == magic1:
    gained_by_magic = gain * (closest_goal // magic2)
    it = closest_goal
    print(it)
  
  a = list(minos[mino_index])
  mino_index = (mino_index + 1) % len(minos)
  
  base_x = 2
  if stored:
    base_y = max([i.y for i in stored]) + 4
  else:
    base_y = 3
    
  a = [i + Point(base_x, base_y) for i in a]
  
  while True:
    aa = moved(a, LEFT if jets[jet_index] == '<' else RIGHT)
    jet_index = (jet_index + 1) % len(jets)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      pass
    else:
      a = aa
      
    aa = moved(a, DOWN)
    
    collision = any([i in stored or i.x < 0 or i.y < 0 or i.x >= width for i in aa])
    if collision:
      stored |= set(a)
      
      break
    else:
      a = aa
  
  
  # if stored:
  #   print(it, max([i.y for i in stored]) + 1 + gained_by_magic)
  it += 1

max([i.y for i in stored]) + 1 + gained_by_magic

  0%|          | 0/1000000 [00:00<?, ?it/s]

999999998428


1560932944615

In [108]:
submit2(_)

Submitting 1560932944615...


True

In [24]:
62 * 7 / 64

6.78125

In [26]:
64 - 9 * 7

1